In [ ]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [ ]:
# Applying Transforms to the Data
image_transforms = { 
    'train': transforms.Compose([
        transforms.Resize(size=224),
        # transforms.RandomResizedCrop(size=300, scale=(0.8, 1.2)),
        # transforms.RandomRotation(degrees=10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        # transforms.Normalize([0.485, 0.456, 0.406],
        #                      [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=224),
        # transforms.Resize(size=256),
        # transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        # transforms.Normalize([0.485, 0.456, 0.406],
        #                      [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        # transforms.Normalize([0.485, 0.456, 0.406],
        #                      [0.229, 0.224, 0.225])
    ])
}

In [ ]:
# Load the Data

# Set train and valid directory paths
dataset=r"D:/alexnet"

train_directory = os.path.join(dataset, 'train_pad')
# dataset = r"C:\Users\ASUS\Inteligencia_Artificial\clasificador"
valid_directory = os.path.join(dataset, 'valid_pad')

# Batch size
bs =10

# Number of classes
num_classes = len(os.listdir(valid_directory))  #10#2#257
print(num_classes)

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=bs, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=bs, shuffle=False)

In [ ]:
import glob
pd.DataFrame([x.replace("\\","/").split("/") for x in glob.glob("D:/alexnet/train_pad/*/*")])[3].value_counts()

In [ ]:
train_data_size, valid_data_size

In [ ]:
alexnet = models.alexnet(pretrained=True)
alexnet

In [ ]:
for param in alexnet.parameters():
    param.requires_grad = True

In [ ]:
alexnet.features[1]= nn.Hardtanh()

In [ ]:
alexnet.classifier[6] = nn.Linear(4096, 4096)
alexnet.classifier.add_module("7",nn.Softplus())
alexnet.classifier.add_module("8", nn.Linear(4096, 4096))
alexnet.classifier.add_module("9",nn.Softplus())
alexnet.classifier.add_module("10", nn.Linear(4096, 2048))
alexnet.classifier.add_module("11", nn.Softplus())
alexnet.classifier.add_module("12", nn.Linear(2048, num_classes))
alexnet.classifier.add_module("13", nn.Softplus())
alexnet.classifier.add_module("14",  nn.LogSoftmax(dim = 1))
alexnet

In [ ]:
summary(alexnet, (3, 224, 224))

In [ ]:
# Define Optimizer and Loss Function
# loss_func = nn.NLLLoss()
#loss_func=nn.MSELoss()
loss_func = nn.CrossEntropyLoss()
# optimizer = optim.RMSprop(lr=0.0001,params=alexnet.parameters())#alexnet.parameters())
optimizer= optim.SGD(alexnet.parameters(), lr=0.0001, momentum=0.9)
optimizer

In [ ]:

def train_and_validate(model, loss_criterion, optimizer, epochs=25,device=0):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in tqdm.tqdm(enumerate(train_data_loader),total=len(train_data_loader)):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in tqdm.tqdm(enumerate(valid_data_loader),total=len(valid_data_loader)):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)
                
                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/valid_data_size 
        avg_valid_acc = valid_acc/valid_data_size

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        #torch.save(model, dataset+'_model_'+str(epoch)+'.pt')
            
    return model, history

In [ ]:
#restore_checkpoint
checkpoint=torch.load(r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell7.pth")
alexnet.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
device = torch.device(0 if torch.cuda.is_available() else "cpu")
# device = torch.device(0)
num_epochs =100


trained_model, history = train_and_validate(alexnet, loss_func, optimizer, num_epochs)

#torch.save(history, dataset+'_history.pt')

In [ ]:
ñ ##breakpoint

In [ ]:
torch.save({    'epoch': 1,
                'model_state_dict':alexnet.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss_func,
                }, r'C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint/best_modell7.pth')

In [ ]:
history = np.array(history)
plt.plot(history[:,0:2])
plt.legend(['Tr Loss', 'Val Loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.ylim(0,1)
plt.savefig(dataset+'_loss_curve.png')
plt.show()

In [ ]:
plt.plot(history[:,2:4])
plt.legend(['Tr Accuracy', 'Val Accuracy'])
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy')
plt.ylim(0,1)
plt.savefig(dataset+'_accuracy_curve.png')
plt.show()

In [ ]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime
from torchsummary import summary
import torch

In [ ]:
weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell7.pth"
num_classes=6
diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)


In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
y_true = []
y_predict =[]
import glob
filenames=glob.glob(r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\train_pad/*/*")
for file in tqdm.tqdm(filenames):
    file=file.replace("\\","/")
    y_true.append(file.split("/")[-2])
    y_predict.append(model_class.predict_file(file,pad=False))
labels=[]
for k,v in diciconario.items():
    labels.append(v)
labels
confusion_mat = confusion_matrix(y_true, y_predict,labels=labels)
conteos=pd.DataFrame(y_true,columns=["clases"]).value_counts()
conteos=conteos.reset_index(drop=False)
matrix=[]
for i,label in enumerate(labels):
    matrix.append(confusion_mat[i]/conteos[conteos["clases"]==label][0].values[0])
import matplotlib.pyplot as plt

plt.imshow(matrix)
matrix,labels

PREDICCIÓN

In [ ]:
#Librerías
import cv2
import torch,torchvision
import numpy as np
import torch.nn as nn
from PIL import Image
from torchvision import models
from torchsummary import summary


Diccionario de clases

In [ ]:
idx_to_class={0: 'area_verde', 1: 'carros', 2: 'casas', 3: 'en_construccion', 4: 'establecimiento', 5: 'multivivienda', 6: 'terreno_baldio'}
# Get a mapping of the indices to the class names, in order to see the ou

In [ ]:
def predict_Alberto(file,model,pad=False):
    x = Image.open(file)
    x = np.asarray(x)
    # if pad:
    #     x=add_pad(x)
    # else:
    x=cv2.resize(x,(224,224))
    x=x.astype("float32")
    x=x/255
    x=np.moveaxis(x,-1,0)
    x = np.expand_dims(x, axis=0)
    img = torch.from_numpy(x).to(device)
    res=list(model(img).cpu().detach().numpy()[0])
    indice=res.index(max(res))
    clase=idx_to_class.get(indice)
    # print(clase)
    return clase 

In [ ]:
import cv2
file=r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\train_pad\multivivienda\75_13.png"
clase=predict_Alberto(file,alexnet)
imagen=cv2.imread(file)
font = cv2.FONT_HERSHEY_SIMPLEX
org = (10, 50)
fontScale = 1
color = (0, 0, 224)
thickness = 2
image = cv2.putText(imagen, clase, org, font, fontScale, color, thickness, cv2.LINE_AA)
cv2.imshow("imagen",imagen)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
alexnet1 = models.alexnet(pretrained=True)
alexnet1

In [ ]:
for param in alexnet1.parameters():
    param.requires_grad = False

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
num_classes=6
alexnet1.classifier[6] = nn.Linear(4096, num_classes)
alexnet1.classifier.add_module("7", nn.LogSoftmax(dim = 1))


In [ ]:
summary(alexnet1, (3, 224, 224))

In [ ]:
file=r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\train\en_construccion\Lerma_1.PNG"

In [ ]:
x = cv2.imread(file)
x=cv2.resize(x,(224,224))
x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
x = np.asarray(x)
x=x.astype("float32")
x=np.moveaxis(x,-1,0)
x = np.expand_dims(x, axis=0)
img = torch.from_numpy(x).to(device=device)
res=list(alexnet(img).cpu().detach().numpy()[0])
indice=res.index(max(res))
clase=idx_to_class.get(indice)
print(clase) 

In [ ]:
alexnet1 = models.alexnet(pretrained=False)
checkpoint=torch.load(r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\ckpoint\best_model.pth")

In [ ]:
device = torch.device(0 if torch.cuda.is_available() else "cpu")

In [ ]:
# predict_Alberto(file=file, model=alexnet1)

In [ ]:
for param in alexnet1.parameters():
    param.requires_grad = False

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
num_classes=6
alexnet1.classifier[6] = nn.Linear(4096, num_classes)
alexnet1.classifier.add_module("7", nn.LogSoftmax(dim = 1))

#alexnet1.eval()

alexnet1

In [ ]:
summary(alexnet1, (3, 224, 224))

In [ ]:
alexnet1.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# file=r"C:\Users\mfpen\Pictures\Clasificador\train\area_verde\605_1.png"
file=r"C:\Users\ASUS\Inteligencia_Artificial\calsificador\train_pad\casas\63_11.png"
import cv2
x = cv2.imread(file)
x=cv2.resize(x,(224,224))
x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
x = np.asarray(x)
x=x.astype("float32")
x=np.moveaxis(x,-1,0)
x = np.expand_dims(x, axis=0)
img = torch.from_numpy(x).to(device)
res=list(alexnet1(img).cpu().detach().numpy()[0])
indice=res.index(max(res))
clase=idx_to_class.get(indice)
print(clase) 